In [ ]:
"""
    Following tutorial: https://www.tensorflow.org/tutorials/structured_data/imbalanced_data
"""

import os

import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

from bclassification.utils_base import print_class_weights
from bclassification.utils_fc import (
    print_dataset,
    obs_to_vect_with_tc,
    obs_vects_to_vect,
    plot_metrics,
    plot_cm,
    plot_roc,
    describe_results,
)
from experience import load_experience
from lib.action_space import is_do_nothing_action
from lib.constants import Constants as Const
from lib.data_utils import make_dir, env_pf, extract_target_windows, moving_window
from lib.dc_opf import TopologyConverter
from lib.run_utils import create_logger
from lib.tf_utils import (
    ResidulaFCBlock,
    MatthewsCorrelationCoefficient,
)
from lib.visualizer import Visualizer, pprint

Visualizer()

experience_dir = make_dir(os.path.join(Const.RESULTS_DIR, "performance-aug"))
results_dir = make_dir(os.path.join(Const.RESULTS_DIR, "bc-fc"))

agent_name = "agent-mip"
case_name = "l2rpn_2019_art"
env_dc = True
verbose = False

case_results_dir = make_dir(os.path.join(results_dir, f"{case_name}-{env_pf(env_dc)}"))
create_logger(logger_name=f"{case_name}-{env_pf(env_dc)}", save_dir=case_results_dir)

case, collector = load_experience(case_name, agent_name, experience_dir, env_dc=env_dc)
obses, actions, rewards, dones = collector.aggregate_data()

pprint("    - Number of chronics:", dones.sum())
pprint("    - Observations:", len(obses))

In [ ]:
"""
    Parameters
"""

random_seed = 0

model_type = "fc"  # "fc" or "res"

n_window_targets = 0
n_window_history = 1
threshold = 0.5

dropout_rate = 0.1
n_hidden = 512

n_batch = 512
n_epochs = 1500

downsampling_rate = 0.05

In [ ]:
"""
    Datasets
"""

np.random.seed(random_seed)
tf.random.set_seed(random_seed)

labels = is_do_nothing_action(actions, case.env).astype(float)
pprint(
    "    - Labels:",
    f"{int(labels.sum())}/{labels.size}",
    "{:.2f} %".format(100 * labels.mean()),
)

mask_positive = extract_target_windows(labels, mask=~dones, n_window=n_window_targets)
mask_negative = np.logical_and(np.random.binomial(1, downsampling_rate, len(labels)), ~mask_positive)
mask_targets = np.logical_or(mask_positive, mask_negative)

pprint(
    "    - Mask (0):",
    mask_negative.sum(),
    "{:.2f} %".format(100 * mask_negative.sum() / mask_targets.sum()),
)
pprint(
    "    - Mask (1):",
    mask_positive.sum(),
    "{:.2f} %".format(100 * mask_positive.sum() / mask_targets.sum()),
)
pprint("    - Mask:", mask_targets.sum())

obs_to_vect = obs_to_vect_with_tc(TopologyConverter(case.env))

X_all = moving_window(
    obses,
    n_window=n_window_history,
    process_fn=obs_to_vect,
    combine_fn=obs_vects_to_vect,
    padding=np.zeros_like(obs_to_vect(obses[0])),
)

X_all = np.vstack(X_all)
Y_all = np.array(labels)

X = np.vstack(X_all[mask_targets, :])
Y = Y_all[mask_targets]

X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.10, random_state=random_seed
)
X_train, X_val, Y_train, Y_val = train_test_split(
    X_train, Y_train, test_size=0.10, random_state=random_seed
)

print_dataset(X, Y, "Data")
print_dataset(X_train, Y_train, "Train")
print_dataset(X_val, Y_val, "Validation")
print_dataset(X_test, Y_test, "Test")

In [ ]:
"""
    Model
"""

metrics = [
    tf.keras.metrics.TruePositives(thresholds=threshold, name="tp"),
    tf.keras.metrics.FalsePositives(thresholds=threshold, name="fp"),
    tf.keras.metrics.TrueNegatives(thresholds=threshold, name="tn"),
    tf.keras.metrics.FalseNegatives(thresholds=threshold, name="fn"),
    tf.keras.metrics.BinaryAccuracy(threshold=threshold, name="accuracy"),
    tf.keras.metrics.Precision(thresholds=threshold, name="precision"),
    tf.keras.metrics.Recall(thresholds=threshold, name="recall"),
    MatthewsCorrelationCoefficient(threshold=threshold, name="mcc"),
]

n_negative, n_positive = np.bincount(Y.astype(int))
n = n_negative + n_positive

class_weight = {0: n / n_negative / 2.0, 1: n / n_positive / 2.0}
initial_bias = np.log([n_positive / n_negative])

print_class_weights(class_weight)
pprint("Initial bias:", "{:.4f}".format(float(initial_bias)))

if model_type == "fc":
    model = tf.keras.Sequential(
        [
            tf.keras.layers.Dense(n_hidden, activation="relu", input_shape=(X.shape[-1],)),
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.Dense(n_hidden, activation="relu"),
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.Dense(n_hidden, activation="relu"),
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.Dense(n_hidden, activation="relu"),
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.Dense(n_hidden, activation="relu"),
            tf.keras.layers.Dense(
                1,
                activation="sigmoid",
                bias_initializer=tf.keras.initializers.Constant(initial_bias),
            ),
        ]
    )
else:
    model = tf.keras.Sequential(
        [
            tf.keras.layers.Dense(
                n_hidden, activation="relu", input_shape=(X.shape[-1],)
            ),
            tf.keras.layers.Dropout(dropout_rate),
            ResidulaFCBlock(n_hidden, activation="relu"),
            tf.keras.layers.Dropout(dropout_rate),
            ResidulaFCBlock(n_hidden, activation="relu"),
            tf.keras.layers.Dropout(dropout_rate),
            ResidulaFCBlock(n_hidden, activation="relu"),
            tf.keras.layers.Dropout(dropout_rate),
            tf.keras.layers.Dense(
                1,
                activation="sigmoid",
                bias_initializer=tf.keras.initializers.Constant(initial_bias),
            ),
        ]
    )

model.compile(
    optimizer=tf.keras.optimizers.Adam(lr=1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=metrics,
)

model_dir = make_dir(os.path.join(case_results_dir, f"model-000-{model_type}"))
checkpoint_path = os.path.join(model_dir, "ckpts")
ckpt = tf.train.Checkpoint(model=model, optimizer=model.optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

pprint("Model directory:", model_dir)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    pprint("Restoring checkpoint from:", ckpt_manager.latest_checkpoint)

In [ ]:
"""
    Training
"""
tensorboard_path = os.path.join(model_dir, "logs")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=tensorboard_path, write_graph=False, write_images=False, update_freq="epoch"
)
pprint("    - TensorBoard cmd:", f"tensorboard --logdir={tensorboard_path}")

training = model.fit(
    X,
    Y,
    epochs=n_epochs,
    batch_size=n_batch,
    class_weight=class_weight,
    validation_data=(X_val, Y_val),
    callbacks=[tensorboard_callback],
    verbose=4,
)

ckpt_save_path = ckpt_manager.save()
pprint(f"    - Saving checkpoint to:", ckpt_save_path)

plot_metrics(training, Y_train, Y_val, save_dir=model_dir)

In [ ]:
"""
    Results
"""

results_train = model.evaluate(X_train, Y_train, batch_size=n_batch, verbose=0)
results_val = model.evaluate(X_val, Y_val, batch_size=n_batch, verbose=0)
results_test = model.evaluate(X_test, Y_test, batch_size=n_batch, verbose=0)
results_all = model.evaluate(X_all, Y_all, batch_size=n_batch, verbose=1)

Y_train_pred = model.predict(X_train, batch_size=n_batch)
Y_val_pred = model.predict(X_val, batch_size=n_batch)
Y_test_pred = model.predict(X_test, batch_size=n_batch)
Y_all_pred = model.predict(X_all, batch_size=n_batch, verbose=1)

describe_results(model.metrics_names, results_train, Y_train, name="Train")
describe_results(model.metrics_names, results_val, Y_val, name="Validation")
describe_results(model.metrics_names, results_test, Y_test, name="Test")
describe_results(model.metrics_names, results_all, Y_all, name="All")

plot_cm(Y_train, Y_train_pred, "Training", save_dir=model_dir)
plot_cm(Y_val, Y_val_pred, "Validation", save_dir=model_dir)
plot_cm(Y_test, Y_test_pred, "Test", save_dir=model_dir)
plot_cm(Y_all, Y_all_pred, "All", save_dir=model_dir)

plot_roc(
    [
        ("Training", Y_train, Y_train_pred),
        ("Validation", Y_val, Y_val_pred),
        ("Test", Y_test, Y_test_pred),
        ("All", Y_all, Y_all_pred),
    ],
    save_dir=model_dir,
)